In [ ]:
#!pip install "adapter-transformers@git+https://github.com/akufeldt/adapter-transformers.git@debug#egg=adapter-transformers&subdirectory=adapter-transformers"

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Softmax

from typing import List, Optional, Tuple, Union, Dict, Any

from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_from_disk, concatenate_datasets
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from transformers import PreTrainedModel, TrainingArguments, Trainer
#from transformers.adapters import AdapterTrainer

import pandas as pd
import numpy as np
import evaluate

import random
import math
import time
from tqdm import tqdm
import os
import json

/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
_numpy_rng = np.random.default_rng(seed)
random.seed(seed)
np.random.seed(seed)
torch.use_deterministic_algorithms(False)
os.environ['PYTHONHASHSEED'] = str(seed)

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load in model

In [5]:
model_name = 'm2m100_418M'
experiment = 'en-ha_finetune_base_model-1'
dataset_name = 'data/en-ha'

In [6]:
model = M2M100ForConditionalGeneration.from_pretrained(f"facebook/{model_name}")
# model = torch.nn.DataParallel(model, device_ids=[2, 3, 4])
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"facebook/{model_name}")

# Prepare data

In [7]:
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [8]:
# Load the datasets from Hugging Face Hub
original_train_dataset = load_dataset("pranjali97/ha-en_RL-grow1_train", split='train')
original_valid_dataset = load_dataset("pranjali97/ha-en_RL-grow1_valid", split='train')  # Assuming the split is also 'train'


In [9]:
# Filter the datasets to only include samples with a score > 0.6
filtered_train_dataset = original_train_dataset.filter(lambda example: example['score'] > 0.6)
filtered_valid_dataset = original_valid_dataset.filter(lambda example: example['score'] > 0.6)

In [10]:
# Concatenate the original and filtered datasets
concatenated_train_dataset = concatenate_datasets([original_train_dataset, filtered_train_dataset])
concatenated_valid_dataset = concatenate_datasets([original_valid_dataset, filtered_valid_dataset])

# Shuffle the datasets
train_dataset = concatenated_train_dataset.shuffle(seed=42)
valid_dataset = concatenated_valid_dataset.shuffle(seed=42)

/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [11]:
# Define the preprocess function
def preprocess_function(examples):
    inputs = examples['src']  # Hausa sentences
    targets = examples['mt']  # English translations
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    # Return the tokenized inputs and labels
    return {'input_ids': model_inputs['input_ids'], 'attention_mask': model_inputs['attention_mask'], 'labels': labels['input_ids']}

# Apply the preprocess function to the datasets
tokenized_train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)
tokenized_valid_dataset = valid_dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=['src', 'ref', 'mt', 'score']  # Specify the correct columns to remove
)

# Create the DatasetDict
tokenized_dataset = DatasetDict({
    'train': tokenized_train_dataset,  # Directly assign the processed dataset
    'validation': tokenized_valid_dataset  # Directly assign the processed dataset
})

Map:   0%|          | 0/32274 [00:00<?, ? examples/s]/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 3675/3675 [00:01<00:00, 3176.68 examples/s]


In [12]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32274
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3675
    })
})

not run

In [ ]:
dataset = DatasetDict({'train':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/cleaned_train.csv')).shuffle(seed=seed),
                        'validation':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/cleaned_dev.csv')).shuffle(seed=seed),
                        'test':Dataset.from_pandas(pd.read_csv(f'{dataset_name}/test.csv')).shuffle(seed=seed),
})

In [ ]:
dataset['test'] = dataset['test'].rename_column('sentence_eng_Latn','en')
dataset['test'] = dataset['test'].rename_column('sentence_hau_Latn','ha')

In [ ]:
dataset

In [ ]:
def preprocess_function(examples):
    inputs = [example for example in examples[src_lang]]
    targets = [example for example in examples[tgt_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True, padding="max_length")
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names['train'])

In [ ]:
tokenized_dataset

# Training Setup

In [12]:
sacrebleu = evaluate.load("sacrebleu")
wer = evaluate.load("wer")

In [13]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    labels = eval_preds.label_ids
    pred_ids = eval_preds.predictions
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]
    
    preds = np.argmax(pred_ids, axis=-1)

    # removeme
    #import warnings
    #warnings.warn(f"unprocessed preds: {preds[0]}\n)")
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) 

    # removeme
    #warnings.warn(f"unprocessed decoded labels: {tokenizer.batch_decode(labels)[0]}\n)")

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # remove me
    #inputs = eval_preds.input_ids
    #decoded_inputs = tokenizer.batch_decode(inputs)
    
    # Removeme
    import warnings
    warnings.warn(f"preds: {decoded_preds[0]}\n)")
    warnings.warn(f"labels: {decoded_labels[0]}\n)")

    bleu_result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    metrics = {"bleu": bleu_result["score"]}

    flattened_decoded_labels = [' '.join([str(x) for x in l]) for l in decoded_labels]
    wer_score = wer.compute(predictions=decoded_preds, references=flattened_decoded_labels)
    metrics["wer"] = wer_score

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    metrics["gen_len"] = np.mean(prediction_lens)
    metrics = {k: round(v, 4) for k, v in metrics.items()}
    return metrics


In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [15]:
training_args = TrainingArguments(
    f"./lang_adapters/{experiment}/model",
    # evaluation_strategy="steps",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    warmup_steps=0,
    # lr_scheduler_type='cosine_with_restarts',
    # gradient_accumulation_steps=4,
    eval_accumulation_steps=16,
    # gradient_checkpointing=True,
    # predict_with_generate=True,
    fp16=True,
    do_train=True,
    do_eval=True,
    logging_steps=5,
    # eval_steps=5,
    save_strategy="epoch",
    metric_for_best_model="bleu",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    #optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 31.28 GiB. GPU 0 has a total capacty of 79.15 GiB of which 28.60 GiB is free. Process 32379 has 2.41 GiB memory in use. Including non-PyTorch memory, this process has 48.12 GiB memory in use. Of the allocated memory 36.96 GiB is allocated by PyTorch, and 10.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Save model
if not os.path.exists(f'./base_model/{experiment}'):
    os.mkdir(f'./base_model/{experiment}')
    
trainer.save_model(f"./base_model/{experiment}")

# Eval finetuned model on test set

In [ ]:
# Evaluate performance
src_lang = 'en'
tgt_lang = 'ha'
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ha"

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained(f"./base_model/{experiment}")
# model = torch.nn.DataParallel(model, device_ids=[2, 3, 4])
model = model.to(device)
tokenizer = M2M100Tokenizer.from_pretrained(f"./base_model/{experiment}")

In [ ]:
test_outputs = trainer.evaluate(tokenized_dataset['test']) #, forced_bos_token_id=tokenizer.get_lang_id("ha")
#test_output_texts = tokenizer.batch_decode(torch.LongTensor(test_outputs.predictions), skip_special_tokens=True)

In [ ]:
test_outputs